In [1]:
from tensorflow import keras

In [2]:
from keras.models import Sequential
from keras.layers import Dense

# Definindo o número de classes que serão previstas
num_classes = 4

# Criando o modelo
modelo = Sequential()
# Adicionando uma camada de entrada com 3 neurônios, uma camada oculta com 4 e as 4 classes de saída
modelo.add(Dense(3, input_dim=3, activation='relu'))
modelo.add(Dense(4, activation='relu'))
modelo.add(Dense(num_classes, activation='softmax'))

# Compilando o modelo
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [4]:
import pandas as pd
df = pd.read_csv('treino_sinais_vitais_com_label.txt', header=None,
                 usecols=[0,3, 4, 5, 6, 7])
df = df.rename(columns={0: 'linha',
                        3: 'qPA',
                        4: 'pulso',
                        5: 'resp',
                        6: 'gravid',
                        7: 'classe'})
df

,linha,qPA,pulso,resp,gravid,classe
0,1,8.416754,75.921057,21.635259,40.000000,2
1,2,8.725890,63.813564,19.718734,41.530427,2
2,3,0.000000,197.210213,19.045471,52.730745,3
3,4,8.733333,143.636181,17.621141,34.679911,2
4,5,1.747626,82.636672,12.209535,69.375882,3
...,...,...,...,...,...,...
1495,1496,4.774128,6.701052,7.380529,24.494467,1
1496,1497,8.400144,81.203081,13.630509,65.385011,3
1497,1498,4.768427,143.261527,21.843486,24.530704,1
1498,1499,1.707746,184.267283,14.256125,61.062546,3


In [7]:
from sklearn.model_selection import train_test_split
#Divisao de testes e treino
x_class = df[['qPA', 'pulso', 'resp']]
y_class = df['classe']
        
xc_train, xc_test, yc_train, yc_test = train_test_split(
    x_class, y_class, test_size=0.2, random_state=30)

In [12]:
import numpy as np
entradas = np.array(x_class)
saida = [1,2,3,4]

modelo.fit(entradas, saida, epochs=10, batch_size=32)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})

In [ ]:
df = pd.read_csv('treino_sinais_vitais_com_label.txt', header=None,
                 usecols=[0,3, 4, 5, 6, 7])
df = df.rename(columns={0: 'linha',
                        3: 'qPA',
                        4: 'pulso',
                        5: 'resp',
                        6: 'gravid',
                        7: 'classe'})

In [ ]:
#Divisao de testes e treino
x_class = df[['qPA', 'pulso', 'resp']]
y_class = df['classe']
        
xc_train, xc_test, yc_train, yc_test = train_test_split(
    x_class, y_class, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [ ]:
## Modelagem
modelo = RandomForestClassifier(random_state=RANDOM_STATE, max_depth=MAX_DEPTH)
modelo.fit(xc_train, yc_train)

y_class_pred = modelo.predict(xc_test)
accuracy = accuracy_score(yc_test, y_class_pred)

print('Acurácia:', accuracy)
print('F1-score: ', f1_score(yc_test, y_class_pred, average='macro'))

In [ ]:

def training(label=0):
    base = reader(label)
    vetor = []

    for b in base:
        values = np.ravel(list(b.values()))
        vetor.append(values)

    def classification():
        df = pd.DataFrame(vetor, columns=['qPA', 'bpm', 'fpm', 'gravidade'])

        df['gravidade'] = pd.cut(df['gravidade'], bins=[0, 25, 50, 75, 100],
                                 labels=['1', '2', '3', '4'], include_lowest=True)

        X = df[['qPA', 'bpm', 'fpm']]
        y = df['gravidade']

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

        modelo = RandomForestClassifier(n_estimators=ESTIMATORS, max_depth=DEPTH, random_state=RANDOM_STATE)
        modelo.fit(X_train, y_train)

        y_pred = modelo.predict(X_test)
        report = classification_report(y_test, y_pred, zero_division=1)

        print('\n\n--CLASSIFIER--\n')

        print(report)

        cm = confusion_matrix(y_test, y_pred)

        cmd = ConfusionMatrixDisplay(
            cm, display_labels=['Crítico', 'Instável', 'p. Estável', 'Estável'])
        cmd.plot()
        plt.savefig('random_forest-matriz_confusão.png')
        
        def predict_gravidade(X):
            test = pd.DataFrame(X)
            test.columns = ['qPA', 'bpm', 'fpm']
            classes = modelo.predict(test)
            return classes[0]

        return predict_gravidade

    def regression():
        df = pd.DataFrame(vetor, columns=['qPA', 'bpm', 'fpm', 'gravidade'])
        X = df[['qPA', 'bpm', 'fpm']]
        y = df['gravidade']

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

        modelo = RandomForestRegressor(n_estimators=ESTIMATORS, max_depth=DEPTH, random_state=RANDOM_STATE)
        modelo.fit(X_train, y_train)

        y_pred = modelo.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        evs = explained_variance_score(y_test, y_pred)
        max_err = max_error(y_test, y_pred)

        print('\n--REGRESSÃO--\n')

        print(f"MSE: {mse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"R²: {r2:.4f}")
        print(f"Explained Variance Score: {evs:.4f}")
        print(f"Max Error: {max_err:.4f}")

        def predict_gravidade(X):
            test = pd.DataFrame(X)
            test.columns = ['qPA', 'bpm', 'fpm']
            gravidade = modelo.predict(test)
            return gravidade[0]

        return predict_gravidade

    return regression(), classification()


def randomForest(samples):

    reg, cla = training()

    samples = np.array(samples)
    response = []
    i = 0

    for sample in samples:
        gravidade = reg(sample.reshape(1, -1))
        classe = cla(sample.reshape(1, -1))
        response.append([i, gravidade, classe])
        i += 1

    with open('random_forest-test_response.txt', 'w') as f:
        for r in response:
            f.write(f'{r[0]}, {r[1]}, {r[2]}\n')